In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('../../../Downloads/tested.csv')

In [3]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,0,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [4]:
def map_gender(x):
    if x == "male":
        return 0
    elif x == "female":
        return 1
    else:
        return x
        
def prepare_data(data:pd.DataFrame):
    data = data[['Survived','Pclass','Sex','Age','SibSp','Parch','Fare']]
    data['Sex'] = data['Sex'].apply(map_gender)
    data['Pclass'].fillna(data['Pclass'].mode(),inplace=True)
    data['Sex'].fillna(data['Sex'].mode(),inplace=True)
    data['Age'].fillna(data['Age'].mean(),inplace=True)
    data['SibSp'].fillna(data['SibSp'].mode(),inplace=True)
    data['Parch'].fillna(data['Parch'].mode(),inplace=True)
    data['Fare'].fillna(data['Fare'].median(),inplace=True)
    Y = data['Survived']
    X = data[['Pclass','Sex','Age','SibSp','Parch','Fare']]
    m,n_x = X.shape
    X = X.T.to_numpy().reshape(n_x,m)
    Y = Y.T.to_numpy().reshape(1,m)
    return X,Y

In [6]:
X,Y = prepare_data(data)
X.shape,Y.shape

/var/folders/l4/hzr9pnz537ld_hkhgw859klm0000gn/T/ipykernel_1268/604371464.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Sex'] = data['Sex'].apply(map_gender)
/var/folders/l4/hzr9pnz537ld_hkhgw859klm0000gn/T/ipykernel_1268/604371464.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Pclass'].fillna(data['Pclass'].mode(),inplace=True)
/var/folders/l4/hzr9pnz537ld_hkhgw859klm0000gn/T/ipykernel_1268/604371464.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

((6, 418), (1, 418))

In [7]:
def initialize_params(n_x,m):
    W = np.random.randn(n_x,1)
    b = np.zeros(1).reshape(1,1)
    parameters = {"W":W,"b":b}
    return parameters

In [8]:
def sigmoid(x):
    sigmoid = 1/(1+np.exp(-x))
    return sigmoid

In [9]:
def forward_propagation(X,parameters):
    W = parameters["W"]
    b = parameters["b"]
    Y_pred = np.dot(W.T,X) + b
    Z = sigmoid(Y_pred)
    return Z

In [10]:
def compute_cost(Y_pred,Y):
    m = Y_pred.shape[1]
    cost = (- 1 / m) * np.sum(Y * np.log(Y_pred) + (1 - Y) * (np.log(1 - Y_pred)))
    cost = np.squeeze(cost)
    return cost

In [12]:
def backward_propagation(parameters,Y_pred,X,Y):
    m = X.shape[1]
    W = parameters["W"]
    b = parameters["b"]
    dW = (1/m) * (np.dot(X,(Y_pred-Y).T))
    db = (1/m) * np.sum((Y_pred-Y),axis=1,keepdims=True)
    # print(db.shape)
    # db = db.reshape(1,m)
    grads = {"dW":dW,"db":db}
    return grads

In [13]:
def update_parameters(parameters,grads, learning_rate=0.001):
    W = parameters["W"]
    b = parameters["b"]
    dW = grads["dW"]
    db = grads["db"]
    W = W - (learning_rate)*dW
    b = b - (learning_rate)*db
    parameters = {"W":W, "b":b}
    return parameters

In [14]:
def logistic_regression(X,Y,num_iterations=10000):
    np.random.seed(1001)
    n_x = X.shape[0]
    m = X.shape[1]
    parameters = initialize_params(n_x,m)
    for iter in range(num_iterations):
        Y_pred = forward_propagation(X,parameters)
        cost = compute_cost(Y_pred,Y)
        grads = backward_propagation(parameters,Y_pred,X,Y)
        parameters = update_parameters(parameters,grads)
        if iter % 1000 == 0:
            print(f"Cost after iteration {iter}: {cost}")
    return parameters

In [15]:
logistic_regressor_params = logistic_regression(X,Y,num_iterations=30000)

Cost after iteration 0: 16.692196789329586
Cost after iteration 1000: 1.1542272037523602
Cost after iteration 2000: 0.9830489792550683
Cost after iteration 3000: 0.845826806983317
Cost after iteration 4000: 0.7370236113042271
Cost after iteration 5000: 0.6506937866447591
Cost after iteration 6000: 0.5815603304526253
Cost after iteration 7000: 0.5254875166150885
Cost after iteration 8000: 0.47947888118817716
Cost after iteration 9000: 0.44130849850573867
Cost after iteration 10000: 0.4092169096783716
Cost after iteration 11000: 0.3818278841340789
Cost after iteration 12000: 0.35810943869728995
Cost after iteration 13000: 0.3373015144005113
Cost after iteration 14000: 0.31884201642085996
Cost after iteration 15000: 0.3023104313161567
Cost after iteration 16000: 0.2873878472806243
Cost after iteration 17000: 0.2738286323689782
Cost after iteration 18000: 0.2614403120404357
Cost after iteration 19000: 0.25006931956044604
Cost after iteration 20000: 0.23959093860677655
Cost after iteration 